In [9]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')

from datetime import timedelta

In [2]:
from config_paper import key_id, secret_key
from cross_signal import *

In [3]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(key_id, secret_key,base_url)

In [4]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-04-30'

In [5]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close'])
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        dataframe['adx'] = tab.ADX(dataframe)

        # required for graphing

        return dataframe

In [6]:
account = api.get_account()
print(account.status)
account.buying_power

ACTIVE


'141284'

In [7]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [8]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.01

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)
trade=True

In [ ]:
trade=True
while trade==True:
    #
    currenthour=time.localtime().tm_hour
    currentminute=time.localtime().tm_min
    currenttime=currenthour*100+currentminute
    
    
    #asset Manager
    assetlist=api.list_positions()
    
    for asset in assetlist:
            #if asset is in list of positions AND is more than 2.4% loss
            # sell and turn that symbol to null
        if float(asset.unrealized_intraday_plpc)< -.024 :
            openPosition = api.get_position(asset.symbol)

            returned = api.submit_order(asset.symbol,int(openPosition.qty),"sell","market","day")
            assetsToTrade[i]= asset.symbol+' nulled for below -2.4% pnl ' +str(currenttime)
            print('Sell', asset.symbol)    
             
            
            
    #buy and sell        
    for i in range(assetListLen) :
        try:
            symbol = assetsToTrade[i]

            returned_data = api.polygon.historic_agg_v2(symbol, 1, 'minute',_from=startdate, to=startdate).df
                #api.get_barset(symbol,barTimeframe,limit=150).df
            data=populateindicators(returned_data)
        except:
            print(symbol)
        
        
        
#buy conditions -------
        try:
            #set last time to buy stocks and when to start buying
            if currenttime <=925 and currenttime >=636:    
            
            
            # Calculated trading indicators
            #populate exponential moving averages




                 # Create the buy signal here
                if data['adx'][-1] > 22 and \
                data['mfi'][-1] < 25 and \
                data['ultosc'][-1] < 25 and \
                data['rsi'][-1] < 33 and \
                data['minusdi'][-1] > 25 and \
                data['minusdi'][-1] > data['plusdi'][-1]:

                    buyord=True
                    assetlist=api.list_positions()
                    for asset in assetlist:
                        # if asset is in the list of positions we have already bought, then do not buy again
                        if asset.symbol == symbol:
                            buyord=False
                            print(asset.symbol,asset.qty)


                    




                    #if asset is not already bought, then buy             
                    if buyord==True:
                        #cashBalance = account.daytrading_buying_power
                        cashBalance = float(account.buying_power)

                        targetPositionSize = int(cashBalance / (data['close'][-1] / positionSizing))

                        returned = api.submit_order(symbol,targetPositionSize,"buy","market","day") # Market order to open position
                        print('Buy ',returned)
        except:
            print(symbol)
                
         
        
        
#Sell conditions ------
        try:
            if data['rsi'][-1] > 71:
                # Closes position
                openPosition = api.get_position(symbol)

                returned = api.submit_order(symbol,int(openPosition.qty),"sell","market","day") # Market order to fully close position
                print('Sell',returned)
        except:
            print(symbol)

        
    
    
    #get current time and shut off trading after 12
     
    if currenthour >= 12: 
        trade=False
        print('Good work, Turning off')
    

    
    

AAP
LNT
AME
AIV
BRKB
BFB
CMS
CTSH
CCI
DOV
EVRG
MKTK
MCK
PFE
RTN
ROL
UTX

AMT
AIV
BRKB
BFB
CMS
CCI
DOV
HBI
KSU
MKTK
MDLZ
PFE
RTN
ROL
UTX

AIV
BRKB
BFB
CCI
DOV
HBI
MKTK
RTN
ROL
UTX

AME
BRKB
BFB
CCI
MKTK
RTN
ROL
UTX

ADBE
AME
BRKB
BFB
MKTK
PFE
RTN
ROL
UTX

BRKB
BFB
CCI
MKTK
PFE
RTN
ROL
UTX

ADBE
AMT
BRKB
BFB
MKTK
PFE
RTN
UTX

Sell Order({   'asset_class': 'us_equity',
    'asset_id': '69b15845-7c63-4586-b274-1cfdfe9df3d8',
    'canceled_at': None,
    'client_order_id': 'f8e2452b-1b1c-4ddf-96c2-1a5196ad0b65',
    'created_at': '2020-04-30T17:05:54.724078Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': 'ba5ce69a-89e2-432f-8a69-2c908b73b765',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '3',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'sell',
    'status': 'accepted',
    'sto

In [10]:
api.close_all_positions()

[Order({   'body': {   'asset_class': 'us_equity',
                 'asset_id': '932a2ac2-9eba-47f1-932a-0ce1408c3683',
                 'canceled_at': None,
                 'client_order_id': '85c399ea-2e94-43c8-8d5e-d5f9e172cec3',
                 'created_at': '2020-04-30T17:14:21.026507Z',
                 'expired_at': None,
                 'extended_hours': False,
                 'failed_at': None,
                 'filled_at': None,
                 'filled_avg_price': None,
                 'filled_qty': '0',
                 'id': '635a2870-ebef-4980-be7f-eb46d43a10b7',
                 'legs': None,
                 'limit_price': None,
                 'order_class': '',
                 'order_type': 'market',
                 'qty': '46',
                 'replaced_at': None,
                 'replaced_by': None,
                 'replaces': None,
                 'side': 'sell',
                 'status': 'accepted',
                 'stop_price': None,
               